In [1]:
NAME1 = "Caroline Andersson" 
NAME2 = "Adam Burman"
GROUP = "84"

In [3]:
### Check Python version
from platform import python_version_tuple
assert python_version_tuple()[:2] == ('3','9'), "You are not running Python 3.9. Make sure to run Python through the course Conda environment."

### Imports

In [46]:
segmented_images_01 = r"C:\Users\carol\deep-machine-learning\project\Dataset\01_labels"
images_01 = r"C:\Users\carol\deep-machine-learning\project\Dataset\01_images"

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import torchvision.transforms

from torch import nn
from torch import optim

### Load Data

Make a dataloader class

In [53]:
from torch.utils.data import Dataset
from itertools import chain
from PIL import Image
import glob
import os

class GTAData(Dataset):
    

    def __init__(self, root, transform):
            """Constructor
        
        Args:
            root (Path/str): Filepath to the data root
            transform (Compose): A composition of image transforms.
        """
            root = Path(root)
            if not (root.exists() and root.is_dir()):
                raise ValueError(f"Data root '{root}' is invalid")
            
            self.root = root
            self.transform = transform
            
       
    def __getitem__(self, index):
        """Get sample by index
        
        Args:
            index (int)
        
        Returns:
             The index'th sample Tensor
        """
        # Access the stored path and label for the correct index
        label, path = 
        # Load the image into memory
        img = Image.open(self.root)
        # Perform transforms, if any.
        if self.transform is not None:
            img = self.transform(img)
        return img
    

      
        

In [54]:
from torchvision.transforms import Compose
transform = Compose([ToTensor()])

example_dataset = GTAData(images_01, transform)
#print(len(example_dataset)) #1
img = example_dataset[1]
print(label) #2

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\carol\\deep-machine-learning\\project\\Dataset\\01_images'

### Augument and Normalize Data, reformat RGB --> Lab where L = X, a,b = Y

###  Create Autoenconder
vgg16 can be used as the encoder by using transfer learning with the gtaV images. Then a decoder has to be constructed from scratch to generate a new image.